In [4]:
# БЛОК 1: ИМПОРТЫ И КАСТОМНЫЕ МЕТРИКИ

import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
import optuna
import shap
import pickle
import warnings
warnings.filterwarnings('ignore')

print("✅ Импорты загружены")

# ==================================================================================
# КАСТОМНАЯ МЕТРИКА WMAE
# ==================================================================================

def wmae_metric(y_true, y_pred, weights):
    """Weighted Mean Absolute Error"""
    return np.average(np.abs(y_true - y_pred), weights=weights)

def wmae_catboost(y_pred, data):
    """WMAE для CatBoost"""
    y_true = data.get_label()
    weights = data.get_weight()
    return 'WMAE', wmae_metric(y_true, y_pred, weights)

def wmae_lgb(y_pred, data):
    """WMAE для LightGBM"""
    y_true = data.get_label()
    weights = data.get_weight()
    return 'WMAE', wmae_metric(y_true, y_pred, weights), False

def wmae_xgb(y_pred, dtrain):
    """WMAE для XGBoost"""
    y_true = dtrain.get_label()
    weights = dtrain.get_weight()
    return 'WMAE', wmae_metric(y_true, y_pred, weights)

print("✅ Кастомные метрики WMAE готовы")

✅ Импорты загружены
✅ Кастомные метрики WMAE готовы


In [5]:
#  БЛОК 2: ЗАГРУЗКА ЧЕКПОИНТА

# ==================================================================================
# ЗАГРУЗКА ПОДГОТОВЛЕННЫХ ДАННЫХ
# ==================================================================================

checkpoint_file = 'data_checkpoint.pkl'

print("📥 Загрузка данных из чекпоинта...")

with open(checkpoint_file, 'rb') as f:
    data = pickle.load(f)

X_final = data['X_final']
X_test_final = data['X_test_final']
X_lgb = data['X_lgb']
X_test_lgb = data['X_test_lgb']
X_xgb = data['X_xgb']
X_test_xgb = data['X_test_xgb']
y_log = data['y_log']
w = data['w']
cat_final = data['cat_final']
test_ids = data['test_ids']

print(f"✅ Данные загружены: {X_final.shape[1]} признаков, {len(X_final)} строк")

📥 Загрузка данных из чекпоинта...
✅ Данные загружены: 162 признаков, 76786 строк


In [10]:
# БЛОК 3: BAYESIAN OPTIMIZATION

# ==================================================================================
# BAYESIAN OPTIMIZATION ДЛЯ ГИПЕРПАРАМЕТРОВ
# ==================================================================================

print("\n🔍 BAYESIAN OPTIMIZATION (50 итераций на модель)...")
print("Это займёт ~30-60 минут, но найдёт оптимальные параметры!\n")

kf_optuna = KFold(n_splits=3, shuffle=True, random_state=42)  # Меньше фолдов для скорости

# ===== CATBOOST OPTIMIZATION =====
def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1500, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0, 5),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'task_type': 'GPU',
        'devices': '0',
        'loss_function': 'RMSE',  # Ближе к WMAE
        'verbose': 0,
        'random_seed': 42,
        'allow_writing_files': False
    }
    
    scores = []
    for train_idx, val_idx in kf_optuna.split(X_final):
        model = CatBoostRegressor(**params)
        model.fit(
            X_final.iloc[train_idx], y_log.iloc[train_idx],
            sample_weight=w.iloc[train_idx],
            cat_features=cat_final,
            eval_set=(X_final.iloc[val_idx], y_log.iloc[val_idx]),
            early_stopping_rounds=100,
            verbose=0
        )
        pred = np.expm1(model.predict(X_final.iloc[val_idx]))
        true = np.expm1(y_log.iloc[val_idx])
        wmae = wmae_metric(true, pred, w.iloc[val_idx])
        scores.append(wmae)
    
    return np.mean(scores)

print("🐱 Оптимизация CatBoost...")
study_cb = optuna.create_study(direction='minimize', study_name='catboost')
study_cb.optimize(objective_catboost, n_trials=20, show_progress_bar=True)
best_params_cb = study_cb.best_params
print(f"   Лучший WMAE: {study_cb.best_value:.2f}")
print(f"   Параметры: {best_params_cb}\n")

# ===== LIGHTGBM OPTIMIZATION =====
def objective_lgb(trial):
    params = {
        'objective': 'mae',  # MAE вместо regression
        'metric': 'mae',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 127),
        'max_depth': trial.suggest_int('max_depth', 8, 15),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'seed': 42,
        'n_jobs': -1
    }
    
    scores = []
    for train_idx, val_idx in kf_optuna.split(X_lgb):
        train_data = lgb.Dataset(X_lgb.iloc[train_idx], y_log.iloc[train_idx], 
                                  weight=w.iloc[train_idx])
        val_data = lgb.Dataset(X_lgb.iloc[val_idx], y_log.iloc[val_idx], 
                                weight=w.iloc[val_idx], reference=train_data)
        
        model = lgb.train(
            params, train_data, 
            num_boost_round=2000,
            valid_sets=[val_data],
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
        )
        
        pred = np.expm1(model.predict(X_lgb.iloc[val_idx]))
        true = np.expm1(y_log.iloc[val_idx])
        wmae = wmae_metric(true, pred, w.iloc[val_idx])
        scores.append(wmae)
    
    return np.mean(scores)

print("💡 Оптимизация LightGBM...")
study_lgb = optuna.create_study(direction='minimize', study_name='lightgbm')
study_lgb.optimize(objective_lgb, n_trials=20, show_progress_bar=True)
best_params_lgb = study_lgb.best_params
print(f"   Лучший WMAE: {study_lgb.best_value:.2f}")
print(f"   Параметры: {best_params_lgb}\n")

# ===== XGBOOST OPTIMIZATION =====
def objective_xgb(trial):
    params = {
        'objective': 'reg:absoluteerror',  # MAE
        'eval_metric': 'mae',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'tree_method': 'gpu_hist',
        'seed': 42
    }
    
    scores = []
    for train_idx, val_idx in kf_optuna.split(X_xgb):
        dtrain = xgb.DMatrix(X_xgb.iloc[train_idx], label=y_log.iloc[train_idx], 
                              weight=w.iloc[train_idx])
        dval = xgb.DMatrix(X_xgb.iloc[val_idx], label=y_log.iloc[val_idx], 
                            weight=w.iloc[val_idx])
        
        model = xgb.train(
            params, dtrain,
            num_boost_round=2000,
            evals=[(dval, 'val')],
            early_stopping_rounds=50,
            verbose_eval=False
        )
        
        pred = np.expm1(model.predict(dval))
        true = np.expm1(y_log.iloc[val_idx])
        wmae = wmae_metric(true, pred, w.iloc[val_idx])
        scores.append(wmae)
    
    return np.mean(scores)

print("⚡ Оптимизация XGBoost...")
study_xgb = optuna.create_study(direction='minimize', study_name='xgboost')
study_xgb.optimize(objective_xgb, n_trials=20, show_progress_bar=True)
best_params_xgb = study_xgb.best_params
print(f"   Лучший WMAE: {study_xgb.best_value:.2f}")
print(f"   Параметры: {best_params_xgb}\n")

# Сохраняем лучшие параметры
best_params = {
    'catboost': best_params_cb,
    'lightgbm': best_params_lgb,
    'xgboost': best_params_xgb
}

with open('best_params_optuna.pkl', 'wb') as f:
    pickle.dump(best_params, f)

print("✅ Оптимизация завершена! Параметры сохранены в 'best_params_optuna.pkl'")

[I 2025-11-29 18:35:34,029] A new study created in memory with name: catboost



🔍 BAYESIAN OPTIMIZATION (50 итераций на модель)...
Это займёт ~30-60 минут, но найдёт оптимальные параметры!

🐱 Оптимизация CatBoost...


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-11-29 18:36:46,108] Trial 0 finished with value: 65751.2582462182 and parameters: {'iterations': 1825, 'learning_rate': 0.03983964846389411, 'depth': 4, 'l2_leaf_reg': 1.5343353120023513, 'random_strength': 1.4928515605716097, 'bagging_temperature': 0.5039218772736574}. Best is trial 0 with value: 65751.2582462182.
[I 2025-11-29 18:39:11,454] Trial 1 finished with value: 66626.18286386097 and parameters: {'iterations': 1519, 'learning_rate': 0.012497716186167771, 'depth': 8, 'l2_leaf_reg': 1.5516399865149908, 'random_strength': 4.763037387425909, 'bagging_temperature': 0.5233921641301655}. Best is trial 0 with value: 65751.2582462182.
[I 2025-11-29 18:41:22,291] Trial 2 finished with value: 65112.05155523218 and parameters: {'iterations': 1659, 'learning_rate': 0.03950711162206714, 'depth': 7, 'l2_leaf_reg': 3.185644576142434, 'random_strength': 2.9556983584188314, 'bagging_temperature': 0.21434975246953492}. Best is trial 2 with value: 65112.05155523218.
[I 2025-11-29 18:45:43

[I 2025-11-29 19:21:53,394] A new study created in memory with name: lightgbm


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[946]	valid_0's l1: 0.419944
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1025]	valid_0's l1: 0.414276
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1470]	valid_0's l1: 0.417318
[I 2025-11-29 19:22:52,676] Trial 0 finished with value: 62265.56278327408 and parameters: {'learning_rate': 0.02354411384047293, 'num_leaves': 124, 'max_depth': 14, 'feature_fraction': 0.7679002176117616, 'bagging_fraction': 0.8792039897732091, 'bagging_freq': 7, 'min_child_samples': 35, 'lambda_l1': 3.664664863194461, 'lambda_l2': 1.098204286612604}. Best is trial 0 with value: 62265.56278327408.
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[619]	valid_0's l1: 0.423829
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:


[I 2025-11-29 19:39:27,693] A new study created in memory with name: xgboost


[I 2025-11-29 19:39:27,690] Trial 19 finished with value: 62187.80016026421 and parameters: {'learning_rate': 0.011984839330405488, 'num_leaves': 111, 'max_depth': 13, 'feature_fraction': 0.6728789330541666, 'bagging_fraction': 0.8276969219042319, 'bagging_freq': 6, 'min_child_samples': 29, 'lambda_l1': 5.425240634451687, 'lambda_l2': 4.163456419631878}. Best is trial 17 with value: 62170.71526181195.
   Лучший WMAE: 62170.72
   Параметры: {'learning_rate': 0.0158614291366624, 'num_leaves': 93, 'max_depth': 14, 'feature_fraction': 0.7006576198474429, 'bagging_fraction': 0.8239219288397926, 'bagging_freq': 6, 'min_child_samples': 30, 'lambda_l1': 2.6556856514238656, 'lambda_l2': 4.290607521856232}

⚡ Оптимизация XGBoost...


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-11-29 19:40:24,210] Trial 0 finished with value: 62496.861326348095 and parameters: {'learning_rate': 0.011633529763994796, 'max_depth': 9, 'min_child_weight': 8, 'subsample': 0.8382566043855415, 'colsample_bytree': 0.6868297542668472, 'gamma': 0.1559836220866534, 'reg_alpha': 2.3039910053801957, 'reg_lambda': 1.7440519502839336}. Best is trial 0 with value: 62496.861326348095.
[I 2025-11-29 19:40:50,872] Trial 1 finished with value: 63590.51931157268 and parameters: {'learning_rate': 0.07158741213442271, 'max_depth': 7, 'min_child_weight': 6, 'subsample': 0.8425427447838256, 'colsample_bytree': 0.7640667380862183, 'gamma': 2.141271639826088, 'reg_alpha': 6.793597699250524, 'reg_lambda': 7.967237807011685}. Best is trial 0 with value: 62496.861326348095.
[I 2025-11-29 19:41:23,809] Trial 2 finished with value: 63390.148536887544 and parameters: {'learning_rate': 0.06255003512687704, 'max_depth': 11, 'min_child_weight': 7, 'subsample': 0.8038250278968775, 'colsample_bytree': 0.8

In [11]:
# БЛОК 4: BAGGING С РАЗНЫМИ SEEDS + OOF ДЛЯ СТЕКИНГА

# ==================================================================================
# ОБУЧЕНИЕ С BAGGING (3 SEEDS) + OUT-OF-FOLD ДЛЯ СТЕКИНГА
# ==================================================================================

print("\n🔄 5-FOLD CV С BAGGING (3 seeds на модель)...")
print("Это создаст OOF предсказания для стекинга\n")

# Загружаем лучшие параметры
with open('best_params_optuna.pkl', 'rb') as f:
    best_params = pickle.load(f)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# OOF предсказания для стекинга
oof_cb = np.zeros(len(X_final))
oof_lgb = np.zeros(len(X_final))
oof_xgb = np.zeros(len(X_final))

# Тестовые предсказания (усредним по seeds и фолдам)
test_preds_cb = np.zeros(len(X_test_final))
test_preds_lgb = np.zeros(len(X_test_lgb))
test_preds_xgb = np.zeros(len(X_test_xgb))

cv_scores_cb = []
cv_scores_lgb = []
cv_scores_xgb = []

BAGGING_SEEDS = [42, 123, 777]  # 3 разных seed для bagging

for fold, (train_idx, val_idx) in enumerate(kf.split(X_final)):
    print(f"\n{'='*60}")
    print(f"FOLD {fold+1}/5")
    print(f"{'='*60}")
    
    X_tr, X_val = X_final.iloc[train_idx], X_final.iloc[val_idx]
    y_tr, y_val = y_log.iloc[train_idx], y_log.iloc[val_idx]
    w_tr, w_val = w.iloc[train_idx], w.iloc[val_idx]
    
    val_true = np.expm1(y_val)
    
    # ===== CATBOOST с BAGGING =====
    preds_cb_seeds = []
    test_cb_seeds = []
    
    for seed in BAGGING_SEEDS:
        params_cb = best_params['catboost'].copy()
        params_cb.update({
            'task_type': 'GPU',
            'devices': '0',
            'loss_function': 'MAE',
            'verbose': 0,
            'random_seed': seed,
            'allow_writing_files': False
        })
        
        model = CatBoostRegressor(**params_cb)
        model.fit(
            X_tr, y_tr, sample_weight=w_tr,
            cat_features=cat_final,
            eval_set=(X_val, y_val),
            early_stopping_rounds=100,
            verbose=0
        )
        
        val_pred = np.expm1(model.predict(X_val))
        test_pred = np.expm1(model.predict(X_test_final))
        
        preds_cb_seeds.append(val_pred)
        test_cb_seeds.append(test_pred)
    
    # Усредняем по seeds
    oof_cb[val_idx] = np.mean(preds_cb_seeds, axis=0)
    test_preds_cb += np.mean(test_cb_seeds, axis=0) / 5
    
    wmae_cb = wmae_metric(val_true, oof_cb[val_idx], w_val)
    cv_scores_cb.append(wmae_cb)
    print(f"   CatBoost WMAE: {wmae_cb:.2f}")
    
    # ===== LIGHTGBM с BAGGING =====
    preds_lgb_seeds = []
    test_lgb_seeds = []
    
    for seed in BAGGING_SEEDS:
        params_lgb = best_params['lightgbm'].copy()
        params_lgb.update({
            'objective': 'mae',
            'metric': 'mae',
            'verbose': -1,
            'seed': seed,
            'n_jobs': -1
        })
        
        train_data = lgb.Dataset(X_lgb.iloc[train_idx], y_tr, weight=w_tr)
        val_data = lgb.Dataset(X_lgb.iloc[val_idx], y_val, weight=w_val, reference=train_data)
        
        model = lgb.train(
            params_lgb, train_data,
            num_boost_round=2000,
            valid_sets=[val_data],
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
        )
        
        val_pred = np.expm1(model.predict(X_lgb.iloc[val_idx]))
        test_pred = np.expm1(model.predict(X_test_lgb))
        
        preds_lgb_seeds.append(val_pred)
        test_lgb_seeds.append(test_pred)
    
    oof_lgb[val_idx] = np.mean(preds_lgb_seeds, axis=0)
    test_preds_lgb += np.mean(test_lgb_seeds, axis=0) / 5
    
    wmae_lgb = wmae_metric(val_true, oof_lgb[val_idx], w_val)
    cv_scores_lgb.append(wmae_lgb)
    print(f"   LightGBM WMAE: {wmae_lgb:.2f}")
    
    # ===== XGBOOST с BAGGING =====
    preds_xgb_seeds = []
    test_xgb_seeds = []
    
    for seed in BAGGING_SEEDS:
        params_xgb = best_params['xgboost'].copy()
        params_xgb.update({
            'objective': 'reg:absoluteerror',
            'eval_metric': 'mae',
            'tree_method': 'gpu_hist',
            'seed': seed
        })
        
        dtrain = xgb.DMatrix(X_xgb.iloc[train_idx], label=y_tr, weight=w_tr)
        dval = xgb.DMatrix(X_xgb.iloc[val_idx], label=y_val, weight=w_val)
        dtest = xgb.DMatrix(X_test_xgb)
        
        model = xgb.train(
            params_xgb, dtrain,
            num_boost_round=2000,
            evals=[(dval, 'val')],
            early_stopping_rounds=50,
            verbose_eval=False
        )
        
        val_pred = np.expm1(model.predict(dval))
        test_pred = np.expm1(model.predict(dtest))
        
        preds_xgb_seeds.append(val_pred)
        test_xgb_seeds.append(test_pred)
    
    oof_xgb[val_idx] = np.mean(preds_xgb_seeds, axis=0)
    test_preds_xgb += np.mean(test_xgb_seeds, axis=0) / 5
    
    wmae_xgb = wmae_metric(val_true, oof_xgb[val_idx], w_val)
    cv_scores_xgb.append(wmae_xgb)
    print(f"   XGBoost  WMAE: {wmae_xgb:.2f}")

print("\n" + "="*60)
print("📊 ИТОГОВЫЕ CV МЕТРИКИ (WMAE):")
print(f"   CatBoost:  {np.mean(cv_scores_cb):.2f} ± {np.std(cv_scores_cb):.2f}")
print(f"   LightGBM:  {np.mean(cv_scores_lgb):.2f} ± {np.std(cv_scores_lgb):.2f}")
print(f"   XGBoost:   {np.mean(cv_scores_xgb):.2f} ± {np.std(cv_scores_xgb):.2f}")
print("="*60)

# Сохраняем OOF и предсказания
stacking_data = {
    'oof_cb': oof_cb,
    'oof_lgb': oof_lgb,
    'oof_xgb': oof_xgb,
    'test_preds_cb': test_preds_cb,
    'test_preds_lgb': test_preds_lgb,
    'test_preds_xgb': test_preds_xgb,
    'y_log': y_log,
    'w': w,
    'cv_scores_cb': cv_scores_cb,
    'cv_scores_lgb': cv_scores_lgb,
    'cv_scores_xgb': cv_scores_xgb
}

with open('stacking_predictions.pkl', 'wb') as f:
    pickle.dump(stacking_data, f)

print("\n✅ OOF предсказания сохранены в 'stacking_predictions.pkl'")


🔄 5-FOLD CV С BAGGING (3 seeds на модель)...
Это создаст OOF предсказания для стекинга


FOLD 1/5


Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


   CatBoost WMAE: 66717.08
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1672]	valid_0's l1: 0.417433
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1165]	valid_0's l1: 0.418308
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1515]	valid_0's l1: 0.41677
   LightGBM WMAE: 64511.36
   XGBoost  WMAE: 64690.39

FOLD 2/5


Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


   CatBoost WMAE: 62232.52
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[2000]	valid_0's l1: 0.412952
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1414]	valid_0's l1: 0.413939
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1476]	valid_0's l1: 0.413023
   LightGBM WMAE: 59845.30
   XGBoost  WMAE: 60295.44

FOLD 3/5


...

   CatBoost WMAE: 65007.14
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1716]	valid_0's l1: 0.416757
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1999]	valid_0's l1: 0.415826
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1977]	valid_0's l1: 0.416445
   LightGBM WMAE: 62021.25
   XGBoost  WMAE: 62638.94

FOLD 4/5


Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


   CatBoost WMAE: 61617.61
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1514]	valid_0's l1: 0.40747
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1794]	valid_0's l1: 0.407856
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1532]	valid_0's l1: 0.405734
   LightGBM WMAE: 58852.68
   XGBoost  WMAE: 59330.19

FOLD 5/5


Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


   CatBoost WMAE: 65087.33
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1079]	valid_0's l1: 0.421079
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1714]	valid_0's l1: 0.420828
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1352]	valid_0's l1: 0.421371
   LightGBM WMAE: 62995.74
   XGBoost  WMAE: 63023.68

📊 ИТОГОВЫЕ CV МЕТРИКИ (WMAE):
   CatBoost:  64132.34 ± 1912.66
   LightGBM:  61645.26 ± 2060.00
   XGBoost:   61995.73 ± 1935.34

✅ OOF предсказания сохранены в 'stacking_predictions.pkl'


In [12]:
# БЛОК 5: СТЕКИНГ — МЕТА-МОДЕЛЬ

# ==================================================================================
# СТЕКИНГ: ОБУЧЕНИЕ МЕТА-МОДЕЛИ
# ==================================================================================

print("\n🏗️ СТЕКИНГ: Обучение мета-модели...")

# Загружаем OOF предсказания
with open('stacking_predictions.pkl', 'rb') as f:
    stack_data = pickle.load(f)

oof_cb = stack_data['oof_cb']
oof_lgb = stack_data['oof_lgb']
oof_xgb = stack_data['oof_xgb']
test_preds_cb = stack_data['test_preds_cb']
test_preds_lgb = stack_data['test_preds_lgb']
test_preds_xgb = stack_data['test_preds_xgb']
y_log = stack_data['y_log']
w = stack_data['w']

# Создаём фичи для мета-модели
X_meta_train = np.column_stack([oof_cb, oof_lgb, oof_xgb])
X_meta_test = np.column_stack([test_preds_cb, test_preds_lgb, test_preds_xgb])

y_meta = np.expm1(y_log)

# Обучаем мета-модель (Ridge с весами)
print("   Обучение Ridge регрессии...")
meta_model = Ridge(alpha=10, random_state=42)
meta_model.fit(X_meta_train, y_meta, sample_weight=w)

# Предсказания мета-модели
meta_train_pred = meta_model.predict(X_meta_train)
meta_test_pred = meta_model.predict(X_meta_test)

# Метрика стекинга
wmae_stacking = wmae_metric(y_meta, meta_train_pred, w)
print(f"   Стекинг WMAE (train): {wmae_stacking:.2f}")

# Веса мета-модели
weights_meta = meta_model.coef_
print(f"\n   Веса мета-модели:")
print(f"      CatBoost:  {weights_meta[0]:.3f}")
print(f"      LightGBM:  {weights_meta[1]:.3f}")
print(f"      XGBoost:   {weights_meta[2]:.3f}")

# Сохраняем мета-модель
with open('meta_model.pkl', 'wb') as f:
    pickle.dump(meta_model, f)

print("\n✅ Мета-модель сохранена в 'meta_model.pkl'")


🏗️ СТЕКИНГ: Обучение мета-модели...
   Обучение Ridge регрессии...
   Стекинг WMAE (train): 67809.73

   Веса мета-модели:
      CatBoost:  0.096
      LightGBM:  0.694
      XGBoost:   0.312

✅ Мета-модель сохранена в 'meta_model.pkl'


In [13]:
# БЛОК 6: ФИНАЛЬНЫЙ ЭКСПОРТ

# ==================================================================================
# ФИНАЛЬНЫЙ ЭКСПОРТ
# ==================================================================================

print("\n🏆 ФИНАЛЬНЫЙ ЭКСПОРТ...")

# Загружаем всё необходимое
with open('data_checkpoint.pkl', 'rb') as f:
    data = pickle.load(f)
    test_ids = data['test_ids']

with open('stacking_predictions.pkl', 'rb') as f:
    preds = pickle.load(f)
    test_preds_cb = preds['test_preds_cb']
    test_preds_lgb = preds['test_preds_lgb']
    test_preds_xgb = preds['test_preds_xgb']

with open('meta_model.pkl', 'rb') as f:
    meta_model = pickle.load(f)

# Стекинг предсказания
X_meta_test = np.column_stack([test_preds_cb, test_preds_lgb, test_preds_xgb])
final_preds_stacking = meta_model.predict(X_meta_test)
final_preds_stacking = np.clip(final_preds_stacking, 20000, None)

# Простой блендинг (для сравнения)
final_preds_blend = 0.3 * test_preds_cb + 0.5 * test_preds_lgb + 0.2 * test_preds_xgb
final_preds_blend = np.clip(final_preds_blend, 20000, None)

# Сохраняем оба варианта
submissions = {
    'stacking': final_preds_stacking,
    'blend': final_preds_blend
}

for name, preds in submissions.items():
    sub = pd.DataFrame({
        'id': test_ids,
        'predict': preds
    })
    
    filename = f'submission_ULTIMATE_{name}.csv'
    sub.to_csv(filename, sep=',', index=False, float_format='%.2f')
    
    print(f"✅ {filename}")
    print(f"   min={preds.min():.0f}, max={preds.max():.0f}, mean={preds.mean():.0f}\n")

print("🎉 ВСЁ ГОТОВО! У тебя теперь ТОП-УРОВЕНЬ ML система!")


🏆 ФИНАЛЬНЫЙ ЭКСПОРТ...
✅ submission_ULTIMATE_stacking.csv
   min=37495, max=1205766, mean=117361

✅ submission_ULTIMATE_blend.csv
   min=20098, max=1096094, mean=92774

🎉 ВСЁ ГОТОВО! У тебя теперь ТОП-УРОВЕНЬ ML система!


In [14]:
# --- ДОБАВИТЬ ЭТО В КОНЕЦ НОУТБУКА ---
import joblib

# 1. Обучаем финальные модели на ВСЕХ данных (X_final)
print("Сохранение финальных моделей для API...")

# CatBoost
final_cb = CatBoostRegressor(**best_params['catboost'])
final_cb.fit(X_final, y_log, sample_weight=w, cat_features=cat_final, verbose=0)
final_cb.save_model("catboost_final.cbm") # CatBoost свой формат любит

# LightGBM
train_data_lgb = lgb.Dataset(X_lgb, y_log, weight=w)
final_lgb = lgb.train(best_params['lightgbm'], train_data_lgb, num_boost_round=2000)
joblib.dump(final_lgb, "lightgbm_final.pkl")

# XGBoost
dtrain_all = xgb.DMatrix(X_xgb, label=y_log, weight=w)
final_xgb = xgb.train(best_params['xgboost'], dtrain_all, num_boost_round=2000)
joblib.dump(final_xgb, "xgboost_final.pkl")

# Meta model (Ridge) уже у тебя есть - meta_model.pkl
# Также нам нужен список колонок, чтобы API знал порядок
cols_info = {
    "cat_features": cat_final,
    "all_columns": list(X_final.columns)
}
joblib.dump(cols_info, "columns_info.pkl")

print("✅ Все модели сохранены для API!")

Сохранение финальных моделей для API...
✅ Все модели сохранены для API!
